# FINAL Meteo 203 Lab Exercise
## Investigating the Onset of the 1997–1998 El Niño: A Zonal Wind and SST Anomaly Analysis  
---

Deadline: 10 December 2025.

### Context
The 1997–1998 El Niño event is one of the strongest on record. Several studies suggest that it was triggered in part by a [strong and sustained Westerly Wind Burst (WWB) in the equatorial Pacific.](https://journals.ametsoc.org/view/journals/clim/34/20/JCLI-D-21-0010.1.xml) In this project, you will explore this by looking at anomalies in zonal wind (u10) and sea surface temperature (SST) across the Pacific Ocean.

### Objectives 
You are tasked to:
1. Download *monthly* [ERA5](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels-monthly-means) or [NCEP/NCAR Reanalysis](https://gdex.ucar.edu/datasets/d090002/) reanalysis data containing u10 and sst. It is up to you to establish your climatologies (suggested: 1950-1996).
    1. ERA5 is easier to work with.
    2. Monthly averaged ERA5 data from 1940 to 2024 is ~3gb total, so you should start downloading as soon as you can. 
3. Calculate monthly or seasonal climatologies (based on multiple years)
4. Compute anomalies for the year 1997
5. Visualize the evolution of both variables in the Pacific leading up to El Niño onset

### Minimum Requirements
By the end of the project, you should produce a Jupyter Notebook containing **at least** 8 Maps
- 4 anomaly maps for u10
    - JFM 1997
    - AMJ 1997
    - JAS 1997
    - OND 1997
- 4 anomaly maps for SST for the same season

Each map should visualize anomalies across the Pacific, ideally highlighting developing patterns.

### Recommended Steps
#### Step 1: Data Preparation
- Download ERA5 or GFS FNL NetCDF data (monthly resolution)
- Suggested variables: `u10`, `sst`
- Clip to Pacific region (`lon`: 120–280 or -160 to 80; `lat`: -30 to 30) to make processing faster

#### Step 2: Climatology
- Group by month or season (e.g., `groupby('valid_time.month')`)
- Calculate the long-term mean for each month or season. You can refer to your old notebooks.

#### Step 3: Anomalies
- Subtract climatology from 1997 values
- For example: `anom = value_1997 - climatology`

#### Step 4: Plotting
- Use `matplotlib.pyplot` or `xarray.plot()`
- **Optional**: use Cartopy to add coastlines or map projection

Label your plots clearly (e.g., “Zonal Wind Anomaly – JJA 1997”)

#### Step 5: Short Discussion
A written Discussion/Conclusion section (5–10 paragraphs) analyzing the anomaly maps, interpreting ENSO‑related signals, and reflecting on how the results relate to the 1997 El Niño onset.
- Summarize the seasonal evolution of wind and SST anomalies
- Identify patterns leading up to the 1997–98 El Niño (e.g., weakening easterlies → westerly bursts → warming SSTs)
- Connect your visualizations to climate processes (e.g., ENSO dynamics, trade winds)
- Discuss whether your anomaly maps support the idea that westerly wind bursts contributed to the 1997 event
- Include reflections on uncertainty or limitations (e.g., regional focus, resolution, seasonal averaging)
- You can also include Niño 3.4 index time-series or figure to support your discussion
- Show your own reasoning and understanding - not just descriptions of plots
- Cite supporting literature
### Expected Output
- A clear and well-documented Jupyter Notebook
- Clean and labeled at least 8 anomaly maps
- A short summary section explaining any patterns you observed

### Rubrics

| **Category**                    | **Criteria**                                                               | **Points** |
| ------------------------------- | -------------------------------------------------------------------------- | ---------- |
| **1. Data Preparation**         | Successfully loads and processes both `u10` and `sst` data                 | 5 pts      |
|                                 | Correctly applies regional and temporal subset (Pacific, 1997, etc.)       | 5 pts      |
| **2. Climatology and Anomaly**  | Correctly calculates climatologies and computes anomalies                  | 5 pts      |
|                                 | Clear logic and code for seasonal aggregation (JFM, AMJ, JAS, OND)              | 5 pts      |
| **3. Map Visualization**        | Produces 8 clear, labeled maps (4 for `u10`, 4 for `sst`)                  | 5 pts      |
|                                 | Proper use of color scales and titles (e.g., diverging cmap for anomalies) | 5 pts      |
| **4. Interpretation & Insight** | Describes patterns or evolution of El Niño indicators                      | 5 pts      |
|                                 | Attempts to connect anomalies with physical processes (e.g., WWBs)         | 5 pts      |
|                                 | Clear discussion with complete references                                  | 5 pts   |
| **5. Notebook Quality**         | Well-structured notebook with markdown explanations and code readability   | 5 pts      |



### Suggested 4-week timeline
| **Week**             | **Date Range**      | **Milestone / Task**                                                                                                                                 |
| -------------------- | ------------------- | ---------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Week 1**           | Nov 12–16 (Wed–Sun) |- **Project briefing** in class<br>- Begin downloading ERA5 monthly data (`u10`, `sst`) |
| **Week 2**           | Nov 17–23 (Mon–Sun) |-  **Preprocessing**: Clean/clip data, compute **monthly climatologies** (1940–1996)<br>- Explore anomalies for JFM, AMJ, JAS, OND 1997                 |
| **Week 3**           | Nov 24–30 (Mon–Sun) |- **Map plotting**: Generate 8 seasonal anomaly maps (4 for `u10`, 4 for `sst`)<br>- Begin interpreting patterns related to El Niño                |
| **Week 4**           | Dec 1–7 (Mon–Sun)   | Write discussion section. Polish notebook: Markdown, labels, titles, captions                                |
| **Final Submission** | **Dec 10 (Wed)** |Submit Jupyter Notebook (`.ipynb`) to GitHub                     |


#### AI Acknowledgement

This exercise was developed for Meteo 203 with the assistance of ChatGPT (OpenAI), particularly in drafting example scripts, refining guide questions, and structuring instructional content. All methodologies were reviewed, tested, and verified by BBR to ensure accuracy and educational alignment.

Students are allowed to use AI tools (e.g., ChatGPT, Copilot) for ideation or code refinement, provided they:

1. Include their own AI Acknowledgement Statement in their submission (e.g., final notebook cell or Markdown section); and

2. Ensure that all outputs reflect their own understanding, coding, and interpretation. AI-generated content must not be blindly copied.